# <font color='red'>STOCK MARKET IS VERY RISKY UNTIL YOU DO IT PROPERLY. PLEASE DO NOT TAKE TRADES JUST BECAUSE THIS TOOL GIVES YOU THE NAME. APPLY YOUR OWN LEARNINGS, CREATE YOUR OWN STRATEGY, ASSESS RISK & TRADE THE PLAN.</font>

In [1]:
'065676'

'065676'

# Imports

In [2]:
# import backtrader
from helpers.datahandler import *
from helpers.investing import *
from helpers.journal_handler import *
from helpers.intraday import *
from helpers.nse_data import *
from helpers.backtest import *

# Defaults

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 25)

In = Investing(check_fresh = True) # Investing or Swing trading analysis
Intra = IntraDay() # Intraday strategies and analysis
NSE = NSEData() # Get NSE Live data
MS = MarketSentiment() # Get Live Market Sentiment TICK, TRIN and 52W high/ low stocks
BT = Backtest()

Checking Fresh Data.....


# Journal
Make a journal in Google Drive for each trade you take to analyse. It is a not so simple process and need to connect with Google API so if you're not comfortable with it, just skip it

In [4]:
private_key = join(expanduser('~'),'Documents','client_secret.json') # Personal for each user. Mine is stored in /home/Documents
JH = JournalHandler(private_key)

journal = JH.get_journal('Finance Journal') # My Google Sheet jornam has name "Finance Journal"

JH.total_pl(journal) # total Profit and Loss up until now

--------------------------------------------------------------------------- 

--------------------------------------------------------------------------- 



/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


609.05

# <font color='teal'>Swing Strategies </font>
## All Details
Basic Details about stocks such as Momentum, Ichimoku, 1-2-3 candles pattern etc

In [5]:
budget = 40000

df = In.get_recent_info(nifty = 100, **{'mvs':[20,50,100,200]})
df.head(2)

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,Date,Name,LTP,Index,CCI Signal,RSI Signal,MACD Signal,Over 20-SMA,Over 50-SMA,Over 100-SMA,Over 200-SMA,Momentum ADX,Ichi Count,1 Candle,2 Candles,3 Candles
0,2021-09-28,ACC,2300.25,Nifty 100,No Signal,No Signal,No Signal,False,False,True,True,23.932572,0,Unknown Red,Bearish Engulfing,Unknown
1,2021-09-28,ABBOTINDIA,21010.80,Nifty 100,No Signal,No Signal,No Signal,True,True,True,True,37.354380,2,Unknown Red,Unknown,Unknown


## 44-SMA 
If you find a `Green` candle taking support on 44 days `Simple Moving Average` line and stock is in uptrend, you can take the trade and set Risk to Reward ratio as 1:2

In [7]:
budget = 40000 # Total Budget
risk = 400 # Risk per trade
df = In.show_full_stats(budget=budget,risk=risk, diff = 151, nifty='nifty_50') # Show only stocks undder your budget and  whose recent candle's lowest is not more than 151 rupees above the moving average line
df

,DATE,OPEN,HIGH,LOW,CLOSE,52W H,52W L,SYMBOL,Diff,CCI Value,RSI Value,MACD Signal,Direction,Ichi,ATR,Triple Candle,Double Candle,Recent Candle,Index,Expected Change %,Max Config Risk
8,2021-09-28,176.85,188.90,176.10,183.95,251.50,154.60,POWERGRID,1.88,249.25,58.856333,No Signal,Undeterministic,1,3.632143,V Pattern,Unknown,Unknown Green,Nifty 50,14.97,396.59
2,2021-09-28,766.70,791.00,754.95,775.10,804.35,452.25,SUNPHARMA,3.29,-29.85,53.204741,No Signal,Rising High,0,14.775000,Unknown,Unknown,Green Doji,Nifty 50,9.49,375.96
0,2021-09-28,127.50,133.65,127.35,131.95,133.65,78.10,NTPC,9.13,152.69,75.396167,No Signal,Rising High,1,3.310714,Unknown,Unknown,Unknown Green,Nifty 50,14.06,395.14
7,2021-09-28,118.70,123.90,118.70,122.95,123.90,73.00,IOC,9.22,176.83,74.026544,No Signal,Rising High,1,2.535714,Unknown,Unknown,Unknown Green,Nifty 50,10.70,398.48
4,2021-09-28,4809.90,4854.10,4755.10,4833.15,5614.60,4135.00,DRREDDY,9.68,-46.99,49.714603,No Signal,Undeterministic,1,80.307143,V Pattern,Unknown,Unknown Green,Nifty 50,5.38,392.06
5,2021-09-28,722.00,733.35,721.50,725.80,775.65,555.95,HDFCLIFE,19.29,-67.37,52.361920,No Signal,Undeterministic,1,13.535714,V Pattern,Unknown,Unknown Green,Nifty 50,4.49,396.04
6,2021-09-28,141.80,143.60,141.00,142.20,143.60,64.10,ONGC,20.10,161.88,77.269819,No Signal,Rising High,1,4.392857,Unknown,Unknown,Inverted Hammer,Nifty 50,8.87,395.63
1,2021-09-28,960.50,977.00,950.80,971.00,997.00,706.50,CIPLA,20.25,124.04,61.424214,No Signal,Rising High,1,17.639286,Unknown,Bullish Engulfing,Unknown Green,Nifty 50,5.75,393.79
3,2021-09-28,168.50,176.30,168.15,174.55,176.30,109.55,COALINDIA,20.33,164.38,75.696443,No Signal,Rising High,1,5.825000,Unknown,Unknown,Unknown Green,Nifty 50,12.79,395.21


## Golden Crossover
When 44 days `SMA` Crosses 200 days `SMA` from below and you find a `Green` candle within 15 days taking support on `44` days line set Risk to Reward ratio as 1:5

In [8]:
for name in In.data['nifty_500']:
    df = In.open_downloaded_stock(name)
    if In.has_golden_crossover(df,lookback=10):
        print(name)

ESCORTS
ITC


## Oversold / Overbought Stocks
Buy when RSI is less than 30 sell when greater than 70

In [9]:
for name in In.data['nifty_100']:
    df = In.open_downloaded_stock(name)
    if In.get_RSI(df) < 30:
        print(name)

## MACD Signal
It is the strategy when MACD (Blue) Line cuts the Signal (Red) line from below then it is a buy signal. When the blue line cuts from upside down, it is sell signal

In [10]:
for name in In.data['nifty_200']:
    df = In.open_downloaded_stock(name)
    if In.macd_signal(df) == 'Buy':
        print(name)

AJANTPHARM
BEL
CANBK
UNIONBANK
VGUARD


## CCI Signal
**Faster than MACD Signal works closely with Stochastic RSI**

When CCI values comes from below  -100 to above -100, buy signal and when it comes from above +100 to below +100, sell signal

In [11]:
for name in In.data['nifty_200']:
    df = In.open_downloaded_stock(name)
    if In.get_CCI(df, signal_only = True) == 'Buy':
        print(name)

DHANI
GODREJCP
HAL
JINDALSTEL
LUPIN
NMDC
SAIL


# <font color='teal'>Intraday Strategies</font>

## Narrow Range 7
If recent candle's range is lowest among 6 other days then the future candle might break it's low/high

In [12]:
for name in In.data['nifty_100']:
    if Intra.NR7_strategy(name):
        print(name)

ADANIGREEN
BANDHANBNK
HINDALCO
ICICIGI
UBL


## Whole Number Open
When Open == Low / High in Whole number as `xxx.00` within 15 minutes

In [35]:
result = Intra.whole_number_strategy(nifty=200, min_val=101,max_val=4999)
for key in result.keys():
    print(key,":\n")
    for r in sorted(result[key],key=lambda x: int(x[-1].split(' ')[-1])):
        print(r,'\n')

Long :

('SBICARD', '1.68%', 'Nifty 100') 

('CADILAHC', '3.96%', 'Nifty 100') 

('OBEROIRLTY', '7.72%', 'Nifty 200') 

('EXIDEIND', '2.44%', 'Nifty 200') 

Short :

('RELIANCE', '-1.94%', 'Nifty 50') 

('AMBUJACEM', '-2.16%', 'Nifty 100') 



## `%` Change probability
Gives the stock names which have the highest probability of providing you `x%` in intraday atleast on both long or short side based on the previous `N` days record

In [36]:
# Gives you top 10 stocks from Nifty 500 which have the highest probability of giving atleast 0.99% on your investment based on the Long/Buy position. Data is based on past 60 trading days
# Test it for different time duration, index, % change and Long / Short criteria based on overall market trend

Intra.prob_by_percent_change(symbol = None, index = 500, time_period = 60, change_percent=0.099, sort_by='Long Probability', top_k = 10) 

{'SUNTECK': {'Long Probability': 0.83, 'Short Probability': 0.63},
 'IDEA': {'Long Probability': 0.83, 'Short Probability': 0.77},
 'LINDEINDIA': {'Long Probability': 0.82, 'Short Probability': 0.7},
 'GUJALKALI': {'Long Probability': 0.78, 'Short Probability': 0.47},
 'HFCL': {'Long Probability': 0.78, 'Short Probability': 0.68},
 'IBREALEST': {'Long Probability': 0.78, 'Short Probability': 0.78},
 'SPARC': {'Long Probability': 0.78, 'Short Probability': 0.68},
 'ADANITRANS': {'Long Probability': 0.77, 'Short Probability': 0.67},
 'BAJAJELEC': {'Long Probability': 0.77, 'Short Probability': 0.68},
 'BRIGADE': {'Long Probability': 0.77, 'Short Probability': 0.73}}

# <font color='Teal'>Live Data</font>


## Market Sentiment: TICK, TRIN (Arm's Index) & 52W high-Low touching shares


In [37]:
MS.get_live_sentiment()

{'Latest Updated on': '29/09/2021 15:19:59',
 'Up': 875,
 'Down': 641,
 'TICK': 234,
 'Volume Up': 2661.8,
 'Volume Down': 754.68,
 'TRIN': 0.387,
 '52W High': 57,
 '52W Low': 10}

## Indices Based
Buy or Sell stock based on which indices are performing highest in first 5 mins. Buy or sell stocks only for those indices which arevperforming the greatest

In [38]:
NSE.current_indices_status(10) # Live as of market is open. works post market also

,index,percentChange,key,variation
48,NIFTY CPSE,4.21,THEMATIC INDICES,95.80
54,NIFTY PSE,3.00,THEMATIC INDICES,120.00
24,NIFTY PSU BANK,2.69,SECTORAL INDICES,64.50
22,NIFTY METAL,2.28,SECTORAL INDICES,126.40
49,NIFTY ENERGY,1.97,THEMATIC INDICES,443.15
23,NIFTY PHARMA,1.65,SECTORAL INDICES,233.75
46,NIFTY COMMODITIES,1.51,THEMATIC INDICES,84.95
27,NIFTY HEALTHCARE INDEX,1.15,SECTORAL INDICES,101.70
45,NIFTY200 MOMENTUM 30,1.14,STRATEGY INDICES,223.95
26,NIFTY REALTY,1.14,SECTORAL INDICES,5.70


In [39]:
NSE.open_nse_index('NIFTY PSE')

,symbol,pChange,open,dayHigh,dayLow,lastPrice,Index
1,OIL,8.94,232.40,265.00,229.00,254.10,Nifty 200
2,NTPC,6.93,131.40,144.25,130.60,141.10,Nifty 50
3,COALINDIA,6.24,174.50,197.00,172.80,185.45,Nifty 50
4,POWERGRID,6.12,183.50,196.30,182.45,195.20,Nifty 50
5,NATIONALUM,6.07,88.10,95.60,88.10,94.40,Nifty 500
6,BHEL,4.06,60.90,63.95,60.40,62.80,Nifty 200
7,IOC,3.82,122.25,127.80,122.00,127.65,Nifty 50
8,NHPC,3.71,28.90,29.40,28.40,29.35,Nifty 500
9,SAIL,3.40,112.40,115.40,110.60,114.10,Nifty 200
10,HINDPETRO,3.05,292.40,300.60,289.00,300.50,Nifty 100


# <font color='teal'>Back Testing</font>
## MACD 

In [40]:
BT.MACD(min_days = 365, nifty = 'nifty_500') # Return top gaining stocks which have traded atleast 365 recent days and are currently in nifty-500

,Total P&L,buys,sells,buy_date,sell_date,hold_period,buy_price,sell_price,p&l,days
DIXON,7070.95,14,13,"[2020-04-03 00:00:00, 2020-05-28 00:00:00, 202...","[2020-05-13 00:00:00, 2020-06-15 00:00:00, 202...","[40, 18, 14, 6, 22, 10, 14, 11, 49, 12, 26, 10...","[3560.0, 4587.0, 5369.0, 5988.0, 6120.0, 7795....","[4600.0, 5000.0, 5600.0, 5899.95, 7499.9, 8133...","[1040.0, 413.0, 231.0, -88.05000000000018, 137...",407
SHREECEM,6566.55,17,17,"[2020-04-13 00:00:00, 2020-05-21 00:00:00, 202...","[2020-05-20 00:00:00, 2020-06-19 00:00:00, 202...","[37, 29, 2, 1, 2, 8, 5, 28, 42, 10, 20, 22, 1,...","[16900.0, 19156.0, 22750.0, 22814.35, 21920.0,...","[18351.05, 21840.0, 22111.0, 22850.0, 21569.0,...","[1451.0499999999993, 2684.0, -639.0, 35.650000...",407
TASTYBITE,4544.5,14,14,"[2020-04-09 00:00:00, 2020-05-29 00:00:00, 202...","[2020-05-11 00:00:00, 2020-07-10 00:00:00, 202...","[32, 42, 5, 7, 13, 58, 14, 27, 8, 15, 24, 9, 9...","[9215.0, 10050.0, 12898.95, 12399.95, 12821.0,...","[10069.95, 12770.0, 11983.6, 12210.0, 12279.95...","[854.9500000000007, 2720.0, -915.3500000000004...",407
BAJFINANCE,3445.9,13,13,"[2020-04-13 00:00:00, 2020-08-26 00:00:00, 202...","[2020-07-16 00:00:00, 2020-09-07 00:00:00, 202...","[94, 12, 7, 1, 42, 1, 20, 34, 7, 12, 11, 28, 4]","[2520.0, 3665.0, 3400.0, 3368.2, 3330.0, 5228....","[3180.0, 3607.0, 3268.0, 3236.35, 4870.0, 5070...","[660.0, -58.0, -132.0, -131.8499999999999, 154...",407
ABBOTINDIA,2919.4,17,17,"[2020-03-31 00:00:00, 2020-04-30 00:00:00, 202...","[2020-04-24 00:00:00, 2020-05-12 00:00:00, 202...","[24, 12, 1, 13, 4, 29, 8, 1, 32, 67, 2, 24, 9,...","[15575.0, 18050.0, 15835.0, 15750.0, 15498.0, ...","[16358.3, 18170.0, 15898.0, 15339.0, 14975.0, ...","[783.2999999999993, 120.0, 63.0, -411.0, -523....",407
SRF,2755.3,16,16,"[2020-04-08 00:00:00, 2020-06-02 00:00:00, 202...","[2020-05-14 00:00:00, 2020-06-17 00:00:00, 202...","[36, 15, 15, 18, 5, 19, 21, 1, 15, 8, 13, 36, ...","[2870.0, 3707.7, 3875.0, 4120.0, 4320.0, 4267....","[3587.0, 3615.0, 3833.9, 4259.9, 4150.55, 4379...","[717.0, -92.69999999999982, -41.09999999999991...",407
BALAMINES,2397.45,15,15,"[2020-06-17 00:00:00, 2020-08-11 00:00:00, 202...","[2020-07-21 00:00:00, 2020-09-02 00:00:00, 202...","[34, 22, 5, 1, 1, 6, 28, 18, 33, 11, 15, 13, 2...","[443.9, 660.0, 872.35, 841.1, 851.35, 850.65, ...","[589.95, 846.0, 829.8, 827.6, 835.0, 826.4, 98...","[146.05000000000007, 186.0, -42.55000000000007...",407
INDIAMART,2210.2,15,15,"[2020-04-07 00:00:00, 2020-05-20 00:00:00, 202...","[2020-05-14 00:00:00, 2020-06-04 00:00:00, 202...","[37, 15, 7, 27, 29, 49, 12, 14, 12, 13, 17, 20...","[2050.0, 2375.0, 2320.0, 2269.8, 3314.95, 5139...","[2139.95, 2344.0, 2172.1, 2978.85, 4694.85, 74...","[89.94999999999982, -31.0, -147.9000000000001,...",407
DIVISLAB,2181.95,13,13,"[2020-04-01 00:00:00, 2020-06-02 00:00:00, 202...","[2020-05-05 00:00:00, 2020-06-12 00:00:00, 202...","[34, 10, 42, 2, 39, 1, 1, 14, 7, 54, 15, 35, 12]","[2000.0, 2500.0, 2200.0, 3192.3, 3210.9, 3789....","[2273.2, 2310.0, 3280.0, 3156.0, 3650.0, 3658....","[273.1999999999998, -190.0, 1080.0, -36.300000...",407
LUXIND,1986.1,10,10,"[2020-04-08 00:00:00, 2020-07-07 00:00:00, 202...","[2020-07-03 00:00:00, 2020-07-15 00:00:00, 202...","[86, 8, 18, 13, 33, 15, 21, 26, 39, 14]","[935.0, 1202.0, 1188.9, 1459.95, 1418.25, 1688...","[1146.8, 1186.0, 1360.0, 1441.0, 1579.95, 1667...","[211.79999999999995, -16.0, 171.0999999999999,...",407


## RSI

In [41]:
BT.RSI(buying_thresh = 30, selling_thresh = 70, min_days = 365, nifty = 'nifty_500') # Give best stocks if they had been bought at 30- RSI and Sold at 70+ andhave traded atleast 365 days

,Total P&L,buys,sells,buy_date,sell_date,hold_period,buy_price,sell_price,p&l,days
TASTYBITE,5583.85,2,2,"[2020-03-25 00:00:00, 2020-09-23 00:00:00]","[2020-06-16 00:00:00, 2020-11-17 00:00:00]","[83, 55]","[7640.15, 10450.0]","[11850.0, 11824.0]","[4209.85, 1374.0]",407
SHREECEM,5009.6,2,2,"[2020-03-02 00:00:00, 2021-08-17 00:00:00]","[2020-11-13 00:00:00, 2021-09-03 00:00:00]","[256, 17]","[22810.0, 26000.0]","[23529.5, 30290.1]","[719.5, 4290.0999999999985]",407
NESTLEIND,3755.25,1,1,[2020-03-16 00:00:00],[2021-08-20 00:00:00],[522],[14800.0],[18555.25],[3755.25],407
MARUTI,2780.55,4,4,"[2020-03-02 00:00:00, 2020-09-25 00:00:00, 202...","[2020-08-10 00:00:00, 2020-12-04 00:00:00, 202...","[161, 70, 99, 33]","[6400.95, 6395.0, 6977.0, 6700.0]","[6690.0, 7798.0, 7317.5, 7448.0]","[289.0500000000002, 1403.0, 340.5, 748.0]",407
BOSCHLTD,2438.0,2,2,"[2020-03-02 00:00:00, 2021-08-18 00:00:00]","[2020-07-08 00:00:00, 2021-09-17 00:00:00]","[128, 30]","[12370.6, 14068.4]","[13490.0, 15387.0]","[1119.3999999999996, 1318.6000000000004]",407
ABBOTINDIA,2350.35,2,2,"[2020-10-30 00:00:00, 2021-01-19 00:00:00]","[2020-12-15 00:00:00, 2021-05-10 00:00:00]","[46, 111]","[15250.0, 14844.65]","[16600.0, 15845.0]","[1350.0, 1000.3500000000004]",407
LAXMIMACH,1726.35,2,2,"[2020-03-12 00:00:00, 2021-04-23 00:00:00]","[2020-06-23 00:00:00, 2021-07-06 00:00:00]","[103, 74]","[2781.65, 5900.0]","[3020.0, 7388.0]","[238.3499999999999, 1488.0]",407
ALKYLAMINE,1611.6,2,2,"[2020-03-13 00:00:00, 2021-05-12 00:00:00]","[2020-04-28 00:00:00, 2021-07-27 00:00:00]","[46, 76]","[1075.0, 3734.0]","[1950.0, 4470.6]","[875.0, 736.6000000000004]",407
BAJFINANCE,1548.85,3,3,"[2020-03-02 00:00:00, 2020-09-25 00:00:00, 202...","[2020-06-24 00:00:00, 2020-11-10 00:00:00, 202...","[114, 46, 132]","[4600.0, 3080.0, 4535.1]","[3125.0, 3898.95, 6740.0]","[-1475.0, 818.9499999999998, 2204.8999999999996]",407
PFIZER,1399.0,2,2,"[2020-03-24 00:00:00, 2021-01-25 00:00:00]","[2020-04-28 00:00:00, 2021-04-15 00:00:00]","[35, 80]","[3850.0, 4825.0]","[4999.0, 5075.0]","[1149.0, 250.0]",407


## CCI

In [42]:
BT.CCI(buying_thresh = -100, selling_thresh = 100, min_days = 365, nifty = 'nifty_50')

,Total P&L,buys,sells,buy_date,sell_date,hold_period,buy_price,sell_price,p&l,days
SHREECEM,16251.7,11,11,"[2020-03-27 00:00:00, 2020-05-21 00:00:00, 202...","[2020-04-24 00:00:00, 2020-06-09 00:00:00, 202...","[28, 19, 23, 9, 22, 22, 16, 21, 46, 33, 20]","[17800.0, 19156.0, 22325.05, 21600.0, 19779.0,...","[18739.0, 21535.95, 21566.0, 21950.0, 20325.0,...","[939.0, 2379.9500000000007, -759.0499999999993...",407
TASTYBITE,5583.85,2,2,"[2020-03-25 00:00:00, 2020-09-23 00:00:00]","[2020-06-16 00:00:00, 2020-11-17 00:00:00]","[83, 55]","[7640.15, 10450.0]","[11850.0, 11824.0]","[4209.85, 1374.0]",407
BAJAJFINSV,3713.1,6,6,"[2020-03-30 00:00:00, 2020-05-21 00:00:00, 202...","[2020-05-05 00:00:00, 2020-06-11 00:00:00, 202...","[36, 21, 6, 76, 20, 21]","[4700.0, 4725.0, 6279.95, 6118.9, 8900.05, 954...","[4890.0, 5200.0, 6400.0, 8750.0, 9144.0, 9600.0]","[190.0, 475.0, 120.05000000000018, 2631.100000...",407
BOSCHLTD,2438.0,2,2,"[2020-03-02 00:00:00, 2021-08-18 00:00:00]","[2020-07-08 00:00:00, 2021-09-17 00:00:00]","[128, 30]","[12370.6, 14068.4]","[13490.0, 15387.0]","[1119.3999999999996, 1318.6000000000004]",407
ABBOTINDIA,2350.35,2,2,"[2020-10-30 00:00:00, 2021-01-19 00:00:00]","[2020-12-15 00:00:00, 2021-05-10 00:00:00]","[46, 111]","[15250.0, 14844.65]","[16600.0, 15845.0]","[1350.0, 1000.3500000000004]",407
DRREDDY,2297.75,7,7,"[2020-03-23 00:00:00, 2020-05-21 00:00:00, 202...","[2020-04-24 00:00:00, 2020-06-03 00:00:00, 202...","[32, 13, 9, 24, 56, 60, 42]","[2680.25, 3950.0, 3915.3, 4400.0, 5056.95, 468...","[4010.0, 3928.8, 4053.55, 5199.95, 5046.95, 45...","[1329.75, -21.199999999999818, 138.25, 799.949...",407
DIVISLAB,2174.45,5,5,"[2020-03-23 00:00:00, 2020-07-13 00:00:00, 202...","[2020-04-21 00:00:00, 2020-08-19 00:00:00, 202...","[29, 37, 46, 72, 15]","[1849.95, 2220.0, 3080.0, 3590.0, 4863.45]","[2295.0, 3166.65, 3442.0, 3790.0, 5084.2]","[445.04999999999995, 946.6500000000001, 362.0,...",407
LAXMIMACH,1726.35,2,2,"[2020-03-12 00:00:00, 2021-04-23 00:00:00]","[2020-06-23 00:00:00, 2021-07-06 00:00:00]","[103, 74]","[2781.65, 5900.0]","[3020.0, 7388.0]","[238.3499999999999, 1488.0]",407
NESTLEIND,1633.9,6,6,"[2020-03-26 00:00:00, 2020-05-27 00:00:00, 202...","[2020-04-21 00:00:00, 2020-07-23 00:00:00, 202...","[26, 57, 76, 84, 26, 35]","[14699.7, 16700.0, 16668.9, 18410.0, 16561.05,...","[16800.0, 17120.0, 15990.05, 16910.0, 17509.0,...","[2100.2999999999993, 420.0, -678.8500000000022...",407
ALKYLAMINE,1611.6,2,2,"[2020-03-13 00:00:00, 2021-05-12 00:00:00]","[2020-04-28 00:00:00, 2021-07-27 00:00:00]","[46, 76]","[1075.0, 3734.0]","[1950.0, 4470.6]","[875.0, 736.6000000000004]",407


# <font color='teal'>Risk Management</font>

In [43]:
# You have a total budget of 40K, and a risk apetite of rs 400 per trade
# You can set manual such as # entry = 3980.1, stop_loss=3780.5

In.get_particulars('NIITLTD',40000,400,entry = 368.5, stop_loss=319)

{'Buying Price': 368.5,
 'Stop-Loss %': 13.43,
 'Target %': 26.85,
 'Quantity': 8.0,
 'Stop-Loss Price': 319,
 'Trigger Price': 467.45,
 'Risk Per Share': 49.5,
 'Profit Per Share': 98.95,
 'Max loss on this config': 396.0,
 'Max Gain on this config': 791.6,
 'Price To Profit Ratio': 3.72,
 'Index': 'Other'}